# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#read the csv the path will be different 

cities_weatherdf = pd.read_csv("../WeatherPy/output_data/cities.csv")

cities_weatherdf.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Chenzhou,25.8000,113.0333,64.76,83,100,6.15,CN,1633660180
1,Los Negros,-17.7333,-63.4333,72.95,63,47,3.38,BO,1633660181
2,Flin Flon,54.7682,-101.8650,44.33,46,5,5.70,CA,1633660183
3,Punta Arenas,-53.1500,-70.9167,44.71,61,0,17.27,CL,1633660185
4,Gawler,-34.6000,138.7333,71.22,31,100,11.14,AU,1633660187


In [3]:
#not exactly sure what it does, but's in all the the class demos, so it probably gets the api kiey from googlwe
gmaps.configure(api_key=g_key)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#the locations and weight for the map
locations = cities_weatherdf[['Lat','Lng']]
humidity = cities_weatherdf['Humidity']

In [5]:
#look at class 3 video  

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
plt.savefig("humid_heatmap")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

A max temperature lower than 80 degrees but higher than 70.


Wind speed less than 10 mph.


Zero cloudiness.


Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.


Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [6]:
#use loc statements to narrow down the resulys
perf_cities_df = cities_weatherdf.loc[(cities_weatherdf['Max Temp'] < 80) & (cities_weatherdf['Max Temp'] > 70) &
                                      (cities_weatherdf['Wind Speed'] < 10) & (cities_weatherdf['Cloudiness'] == 0)] 

perf_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
70,Broken Hill,-31.9500,141.4333,73.11,17,0,5.75,AU,1633660315
236,San Patricio,28.0170,-97.5169,79.84,57,0,7.45,US,1633660629
345,Sirte,31.2089,16.5887,74.88,72,0,8.14,LY,1633660847
360,Tūkrah,32.5341,20.5791,71.60,56,0,6.17,LY,1633660876
529,Alanya,36.5438,31.9998,71.29,31,0,4.88,TR,1633661204


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = perf_cities_df.copy()

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
70,Broken Hill,-31.9500,141.4333,73.11,17,0,5.75,AU,1633660315
236,San Patricio,28.0170,-97.5169,79.84,57,0,7.45,US,1633660629
345,Sirte,31.2089,16.5887,74.88,72,0,8.14,LY,1633660847
360,Tūkrah,32.5341,20.5791,71.60,56,0,6.17,LY,1633660876
529,Alanya,36.5438,31.9998,71.29,31,0,4.88,TR,1633661204


In [8]:
hotel_df["Hotel Name"] = ""


In [9]:
#get the base url (glasses)
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
# also rewatch class video 2 and 3 for api
#loop through the rows of the df

for index, row in hotel_df.iterrows():
    
    coordinates = f"{row['Lat']},{row['Lng']}"
    params = {"location": coordinates, "radius": 5000, "type": 'lodging', "key": g_key}
    response = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except(KeyError, IndexError):
        print('Missing Field. Skipping...')

In [10]:
#check if the hotels got added
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
70,Broken Hill,-31.9500,141.4333,73.11,17,0,5.75,AU,1633660315,Royal Exchange Hotel
236,San Patricio,28.0170,-97.5169,79.84,57,0,7.45,US,1633660629,"Motel 6 Sinton, TX"
345,Sirte,31.2089,16.5887,74.88,72,0,8.14,LY,1633660847,الهادي ابوشعاله سرت
360,Tūkrah,32.5341,20.5791,71.60,56,0,6.17,LY,1633660876,Lagrota restaurant
529,Alanya,36.5438,31.9998,71.29,31,0,4.88,TR,1633661204,Panorama Hotel


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
plt.savefig("perf_hotels")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>